In [41]:
import boto3
import pandas as pd
import time
import os
import json
import urllib
from datetime import date
import seaborn as sns
import numpy as np

In [2]:
aws_access_key_id = '------------------------'
aws_secret_access_key= '--------------------------'
region_name='ap-southeast-2'

In [3]:
session = boto3.session.Session(aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_access_key,
                               region_name=region_name)

In [1]:
# s3 = boto3.client('s3')

In [2]:
# s3.list_buckets()['Buckets']

In [3]:
# s3.list_buckets()['Buckets'][1]

In [4]:
s3_resource = boto3.resource('s3')

audio_list = []
job_list = []

for object in s3_resource.Bucket('inputbucketname').objects.all():
    if object.key[-3:] == 'mp3':
        print(object.key[0:len(object.key)-4])
        audio_list.append(object.key)
        job_list.append(object.key[0:len(object.key)-4])
        
df_audio_files = pd.DataFrame({'file_name':audio_list,
                              'job_name':job_list})   
df_audio_files

## Transcribe Job

In [39]:
transcribe_client = boto3.client('transcribe')

st_time = time.time()


def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp3',
        LanguageCode='en-US'
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                response = urllib.request.urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri'])
                data = json.loads(response.read())
                text = data['results']['transcripts'][0]['transcript']
                print("========== below is output of speech-to-text ========================")
                print(text)
                print("=====================================================================")
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)


# def main():
for i in range(df_audio_files.shape[0]):
    file_uri = 's3://inputbucketname/'+df_audio_files.iloc[i,0]
    job_name = df_audio_files.iloc[i,1]
    transcribe_file(job_name, file_uri, transcribe_client)
#     return


# if __name__ == '__main__':
#     main()

    
print('processing time = %s' %(time.time()-st_time))    

Waiting for samples_mp3_ted_speakers_BillGates_sample-1. Current status is IN_PROGRESS.
Waiting for samples_mp3_ted_speakers_BillGates_sample-1. Current status is IN_PROGRESS.
Job samples_mp3_ted_speakers_BillGates_sample-1 is COMPLETED.
========== below is output of speech-to-text ========================
He said the same phrase 30 times.
Waiting for samples_mp3_ted_speakers_BillGates_sample-2. Current status is IN_PROGRESS.
Waiting for samples_mp3_ted_speakers_BillGates_sample-2. Current status is IN_PROGRESS.
Job samples_mp3_ted_speakers_BillGates_sample-2 is COMPLETED.
========== below is output of speech-to-text ========================
Pluck the bright rose without leaves.
Waiting for samples_mp3_ted_speakers_BillGates_sample-3. Current status is IN_PROGRESS.
Waiting for samples_mp3_ted_speakers_BillGates_sample-3. Current status is IN_PROGRESS.
Job samples_mp3_ted_speakers_BillGates_sample-3 is COMPLETED.
========== below is output of speech-to-text ========================
two 